# 02: Basic Operations and Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/02_basic_operations_and_analysis.ipynb)

Now that you understand xsnow's data structure, let's learn how to work with the data: selecting, filtering, and performing basic analyses.

## What You'll Learn

- Selecting data by location, time, and layer
- Filtering data with conditions
- Computing profile-level summaries
- Time series operations


## Installation (For Colab Users)

If you're using Google Colab, run the cell below to install xsnow and dependencies. If you're running locally and have already installed xsnow, you can skip this cell.


In [1]:

%pip install -q numpy pandas xarray matplotlib seaborn dask netcdf4
%pip install -q git+https://gitlab.com/avacollabra/postprocessing/xsnow



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.5/377.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 14.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have 

## Setup: Load Sample Data

Let's load a sample profile timeseries dataset to work with. This dataset has 5 locations, 416 times, 5 slopes, 1 realization, and 33 layers.

xsnow.read can be passed a list of paths or a directory.


In [6]:
import xsnow
import numpy as np

# Load sample data
datapath = xsnow.sample_data.snp_gridded_dir()

# Read all files within one directory of sample data
ds = xsnow.read(f"{datapath}/pros/gridded/")
ds



<xsnowDataset>
  Locations:  5
  Timestamps: 416 (2024-01-16--2024-02-02)
  Profiles:   NotImplementedYet

    employing the <xarray.Dataset> Size: 42MB
    Dimensions:                   (location: 5, time: 416, slope: 5,
                                   realization: 1, layer: 33)
    Coordinates:
      * location                  (location) <U5 100B 'VIR1A' 'VIR2A' ... 'VIR5A'
      * time                      (time) datetime64[ns] 3kB 2024-01-16T05:00:00 ....
      * slope                     (slope) int64 40B 0 1 2 3 4
      * realization               (realization) int64 8B 0
      * layer                     (layer) int64 264B 0 1 2 3 4 5 ... 28 29 30 31 32
        inclination               (location, slope) int64 200B 0 38 38 ... 38 38 38
        azimuth                   (location, slope) int64 200B 0 0 90 ... 90 180 270
        latitude                  (location) float64 40B 47.15 47.44 ... 47.44 47.37
        longitude                 (location) float64 40B 11.19 11.29 11.3 11.39 11.5
        altitude                  (location) float64 40B 2.372e+03 ... 2.066e+03
        z                         (location, time, slope, realization, layer) float32 1MB ...
    Data variables: (12/90)
        air_content               (location, time, slope, realization, layer) float32 1MB ...
        bond_size                 (location, time, slope, realization, layer) float32 1MB ...
        coord_number              (location, time, slope, realization, layer) float32 1MB ...
        dendricity                (location, time, slope, realization, layer) float32 1MB ...
        density                   (location, time, slope, realization, layer) float32 1MB ...
        element_ID                (location, time, slope, realization, layer) float32 1MB ...
        ...                        ...
        Ss                        (location, slope, realization, time) float64 83kB ...
        zS4                       (location, slope, realization, time) float64 83kB ...
        S4                        (location, slope, realization, time) float64 83kB ...
        zS5                       (location, slope, realization, time) float64 83kB ...
        S5                        (location, slope, realization, time) float64 83kB ...
        HS                        (location, time, slope, realization) float32 42kB ...
    Attributes:
        Conventions:  CF-1.8
        title:        Snowpack Profile for VIR1A
        source:       SNOWPACK model output (.pro file)
        history:      2025-11-13T18:54:08.878715+00:00: Created by xsnow.
        crs:          EPSG:4326

## Part 1: Selecting Data

You can select parts of your data in two ways:

### Selecting by Label with `.sel()`

Use `.sel()` when you know the label (like a date or location name):

**What you'll see:** The code below selects data for a specific location and time. The result is a smaller dataset containing only the selected data.


In [8]:
# Show the first 10 time dimensions
ds.time[:10]


<xarray.DataArray 'time' (time: 10)> Size: 80B
array(['2024-01-16T05:00:00.000000000', '2024-01-16T06:00:00.000000000',
       '2024-01-16T07:00:00.000000000', '2024-01-16T08:00:00.000000000',
       '2024-01-16T09:00:00.000000000', '2024-01-16T10:00:00.000000000',
       '2024-01-16T11:00:00.000000000', '2024-01-16T12:00:00.000000000',
       '2024-01-16T13:00:00.000000000', '2024-01-16T14:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 80B 2024-01-16T05:00:00 ... 2024-01-16T14:...

In [9]:
# Select data based on a time dimension
ds.sel(time='2024-01-16T05')

<xsnowDataset>
  Locations:  5
  Timestamps: 0 (N/A--N/A)
  Profiles:   NotImplementedYet

    employing the <xarray.Dataset> Size: 103kB
    Dimensions:                   (location: 5, slope: 5, realization: 1, layer: 33)
    Coordinates:
      * location                  (location) <U5 100B 'VIR1A' 'VIR2A' ... 'VIR5A'
      * slope                     (slope) int64 40B 0 1 2 3 4
      * realization               (realization) int64 8B 0
      * layer                     (layer) int64 264B 0 1 2 3 4 5 ... 28 29 30 31 32
        time                      datetime64[ns] 8B 2024-01-16T05:00:00
        inclination               (location, slope) int64 200B 0 38 38 ... 38 38 38
        azimuth                   (location, slope) int64 200B 0 0 90 ... 90 180 270
        latitude                  (location) float64 40B 47.15 47.44 ... 47.44 47.37
        longitude                 (location) float64 40B 11.19 11.29 11.3 11.39 11.5
        altitude                  (location) float64 40B 2.372e+03 ... 2.066e+03
        z                         (location, slope, realization, layer) float32 3kB ...
    Data variables: (12/90)
        air_content               (location, slope, realization, layer) float32 3kB ...
        bond_size                 (location, slope, realization, layer) float32 3kB ...
        coord_number              (location, slope, realization, layer) float32 3kB ...
        dendricity                (location, slope, realization, layer) float32 3kB ...
        density                   (location, slope, realization, layer) float32 3kB ...
        element_ID                (location, slope, realization, layer) float32 3kB ...
        ...                        ...
        Ss                        (location, slope, realization) float64 200B 6.0...
        zS4                       (location, slope, realization) float64 200B 0.0...
        S4                        (location, slope, realization) float64 200B 6.0...
        zS5                       (location, slope, realization) float64 200B 0.0...
        S5                        (location, slope, realization) float64 200B nan...
        HS                        (location, slope, realization) float32 100B nan...
    Attributes:
        Conventions:  CF-1.8
        title:        Snowpack Profile for VIR1A
        source:       SNOWPACK model output (.pro file)
        history:      2025-11-13T18:54:08.878715+00:00: Created by xsnow.
        crs:          EPSG:4326

### Selecting by Index with `.isel()`

Use `.isel()` when you want to select by position (first, second, etc.):

**What you'll see:** These examples show how to select the first time step, first few time steps, surface layer, and multiple layers. The resulting datasets will have fewer dimensions than the original.


In [10]:
# Select the 10th time dimension
ds.isel(time=10)

<xsnowDataset>
  Locations:  5
  Timestamps: 0 (N/A--N/A)
  Profiles:   NotImplementedYet

    employing the <xarray.Dataset> Size: 103kB
    Dimensions:                   (location: 5, slope: 5, realization: 1, layer: 33)
    Coordinates:
      * location                  (location) <U5 100B 'VIR1A' 'VIR2A' ... 'VIR5A'
      * slope                     (slope) int64 40B 0 1 2 3 4
      * realization               (realization) int64 8B 0
      * layer                     (layer) int64 264B 0 1 2 3 4 5 ... 28 29 30 31 32
        time                      datetime64[ns] 8B 2024-01-16T15:00:00
        inclination               (location, slope) int64 200B 0 38 38 ... 38 38 38
        azimuth                   (location, slope) int64 200B 0 0 90 ... 90 180 270
        latitude                  (location) float64 40B 47.15 47.44 ... 47.44 47.37
        longitude                 (location) float64 40B 11.19 11.29 11.3 11.39 11.5
        altitude                  (location) float64 40B 2.372e+03 ... 2.066e+03
        z                         (location, slope, realization, layer) float32 3kB ...
    Data variables: (12/90)
        air_content               (location, slope, realization, layer) float32 3kB ...
        bond_size                 (location, slope, realization, layer) float32 3kB ...
        coord_number              (location, slope, realization, layer) float32 3kB ...
        dendricity                (location, slope, realization, layer) float32 3kB ...
        density                   (location, slope, realization, layer) float32 3kB ...
        element_ID                (location, slope, realization, layer) float32 3kB ...
        ...                        ...
        Ss                        (location, slope, realization) float64 200B 6.0...
        zS4                       (location, slope, realization) float64 200B 0.0...
        S4                        (location, slope, realization) float64 200B 6.0...
        zS5                       (location, slope, realization) float64 200B 0.0...
        S5                        (location, slope, realization) float64 200B nan...
        HS                        (location, slope, realization) float32 100B nan...
    Attributes:
        Conventions:  CF-1.8
        title:        Snowpack Profile for VIR1A
        source:       SNOWPACK model output (.pro file)
        history:      2025-11-13T18:54:08.878715+00:00: Created by xsnow.
        crs:          EPSG:4326

ds.sel(time='2024-01-16T05') is saying to xsnow to "Find the element whose coordinate value along the time dimension is exactly '2024-01-16T05'."

*   This is looking for an actual value along the coordinate array.

ds.isel(time=10) is saying to xsnow to "Find the 10th element by index along the time dimension."

* This is a positional selection and does not care what the coordinate value or the time value is at the point.

# This can be done for other variables as well. Below we will see selection based on specific locations, specific slopes and a time range.

In [15]:
# Show locations in the ds
locations = ds.coords['location'].values
print(locations)

# Show all profiles from the VIR2A location
VIR2A = ds.sel(location='VIR2A')
print(VIR2A)


['VIR1A' 'VIR2A' 'VIR3A' 'VIR4A' 'VIR5A']
<xsnowDataset>
  Locations:  0
  Timestamps: 416 (2024-01-16--2024-02-02)
  Profiles:   NotImplementedYet

    employing the <xarray.Dataset> Size: 8MB
    Dimensions:                   (time: 416, slope: 5, realization: 1, layer: 33)
    Coordinates:
      * time                      (time) datetime64[ns] 3kB 2024-01-16T05:00:00 ....
      * slope                     (slope) int64 40B 0 1 2 3 4
      * realization               (realization) int64 8B 0
      * layer                     (layer) int64 264B 0 1 2 3 4 5 ... 28 29 30 31 32
        location                  <U5 20B 'VIR2A'
        inclination               (slope) int64 40B 0 38 38 38 38
        azimuth                   (slope) int64 40B 0 0 90 180 270
        latitude                  float64 8B 47.44
        longitude                 float64 8B 11.29
        altitude                  float64 8B 1.749e+03
        z                         (time, slope, realization, layer) float32 

isel is zero based positional indexing. For the following array of locations:
['VIR1A' 'VIR2A' 'VIR3A' 'VIR4A' 'VIR5A']

* isel(location=0) → 'VIR1A' (1st location)
* isel(location=1) → 'VIR2A' (2nd location)
* isel(location=2) → 'VIR3A' (3rd location)



In [24]:
# Select by position VIR2A
VIR2A = ds.isel(location=1)
print(VIR2A)

<xsnowDataset>
  Locations:  0
  Timestamps: 416 (2024-01-16--2024-02-02)
  Profiles:   NotImplementedYet

    employing the <xarray.Dataset> Size: 8MB
    Dimensions:                   (time: 416, slope: 5, realization: 1, layer: 33)
    Coordinates:
      * time                      (time) datetime64[ns] 3kB 2024-01-16T05:00:00 ....
      * slope                     (slope) int64 40B 0 1 2 3 4
      * realization               (realization) int64 8B 0
      * layer                     (layer) int64 264B 0 1 2 3 4 5 ... 28 29 30 31 32
        location                  <U5 20B 'VIR2A'
        inclination               (slope) int64 40B 0 38 38 38 38
        azimuth                   (slope) int64 40B 0 0 90 180 270
        latitude                  float64 8B 47.44
        longitude                 float64 8B 11.29
        altitude                  float64 8B 1.749e+03
        z                         (time, slope, realization, layer) float32 275kB ...
    Data variables: (12/90)
    

In [17]:
# Show all slopes in the ds
slopes = ds.coords['slope'].values
print(slopes)

# Show attributes of the 2 slope
slope2 = ds.sel(slope=2)
print(slope2)

[0 1 2 3 4]
<xsnowDataset>
  Locations:  5
  Timestamps: 416 (2024-01-16--2024-02-02)
  Profiles:   NotImplementedYet

    employing the <xarray.Dataset> Size: 8MB
    Dimensions:                   (location: 5, time: 416, realization: 1, layer: 33)
    Coordinates:
      * location                  (location) <U5 100B 'VIR1A' 'VIR2A' ... 'VIR5A'
      * time                      (time) datetime64[ns] 3kB 2024-01-16T05:00:00 ....
      * realization               (realization) int64 8B 0
      * layer                     (layer) int64 264B 0 1 2 3 4 5 ... 28 29 30 31 32
        slope                     int64 8B 2
        inclination               (location) int64 40B 38 38 38 38 38
        azimuth                   (location) int64 40B 90 90 90 90 90
        latitude                  (location) float64 40B 47.15 47.44 ... 47.44 47.37
        longitude                 (location) float64 40B 11.19 11.29 11.3 11.39 11.5
        altitude                  (location) float64 40B 2.372e+03 .

In [29]:
# Determine the azimuth of the 2 slopes for all locations
slope2_azimuth = ds.sel(slope=2).coords['azimuth']
print(slope2_azimuth)

<xarray.DataArray 'azimuth' (location: 5)> Size: 40B
array([90, 90, 90, 90, 90])
Coordinates:
  * location     (location) <U5 100B 'VIR1A' 'VIR2A' 'VIR3A' 'VIR4A' 'VIR5A'
    slope        int64 8B 2
    inclination  (location) int64 40B 38 38 38 38 38
    azimuth      (location) int64 40B 90 90 90 90 90
    latitude     (location) float64 40B 47.15 47.44 47.15 47.44 47.37
    longitude    (location) float64 40B 11.19 11.29 11.3 11.39 11.5
    altitude     (location) float64 40B 2.372e+03 1.749e+03 ... 2.066e+03


In [34]:
# Select the last time step, third location, and first slope
subset = ds.isel(time=-1, location=2, slope=0)
print(subset)

<xsnowDataset>
  Locations:  0
  Timestamps: 0 (N/A--N/A)
  Profiles:   NotImplementedYet

    employing the <xarray.Dataset> Size: 4kB
    Dimensions:                   (realization: 1, layer: 33)
    Coordinates:
      * realization               (realization) int64 8B 0
      * layer                     (layer) int64 264B 0 1 2 3 4 5 ... 28 29 30 31 32
        time                      datetime64[ns] 8B 2024-02-02T12:00:00
        slope                     int64 8B 0
        location                  <U5 20B 'VIR3A'
        inclination               int64 8B 0
        azimuth                   int64 8B 0
        latitude                  float64 8B 47.15
        longitude                 float64 8B 11.3
        altitude                  float64 8B 1.86e+03
        z                         (realization, layer) float32 132B -8.68 ... nan
    Data variables: (12/90)
        air_content               (realization, layer) float32 132B 73.0 ... nan
        bond_size                 (real

In [37]:
# Determine the density of the bottom layer for the last time step, third location, first slope
bottom_density = subset['density'].isel(layer=0)
print(bottom_density)

<xarray.DataArray 'density' (realization: 1)> Size: 4B
array([246.1], dtype=float32)
Coordinates:
  * realization  (realization) int64 8B 0
    time         datetime64[ns] 8B 2024-02-02T12:00:00
    layer        int64 8B 0
    slope        int64 8B 0
    location     <U5 20B 'VIR3A'
    inclination  int64 8B 0
    azimuth      int64 8B 0
    latitude     float64 8B 47.15
    longitude    float64 8B 11.3
    altitude     float64 8B 1.86e+03
    z            (realization) float32 4B -8.68
Attributes:
    standard_name:  mass_density_of_snow
    units:          kg m-3
    long_name:      mass density of snow


In [41]:
# Determine the density of the surface layer for the same subset
surface_density = subset['density'].isel(layer=-1)
print(surface_density.item())

nan


This returns nan? This is happening because when selecting the last variable in the array by index (-1), it is selecting a layer not filled with snow at this moment in time, used later in the dataset.

In [44]:
# Assess all layer densities for this subset
layer_densities = subset['density']
print(layer_densities.values)

[[246.1 251.  210.1 109.   83.2 121.2   nan   nan   nan   nan   nan   nan
    nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
    nan   nan   nan   nan   nan   nan   nan   nan   nan]]


In [54]:
# Find the last layer before z is nan
surface_layer = layer_densities.where(layer_densities.notnull(), drop=True).isel(layer=-1)
print(surface_layer.item())

121.19999694824219


### Selecting by Depth (using 'z' coordinate)

Remember the `z` coordinate? We can use it to select layers by depth:

**What you'll see:** The code below filters for layers within 50 cm of the surface. The result will have fewer layers than the original dataset, containing only layers that meet the depth criteria.


In [55]:
# Select layers within 50 cm of surface (z between 0 and -0.5)
# Note: z is negative downward, so we want z >= -0.5
shallow = ds.where((ds.coords['z'] >= -0.5) & (ds.coords['z'] <= 0), drop=True)
shallow

<xsnowDataset>
  Locations:  5
  Timestamps: 373 (2024-01-18--2024-02-02)
  Profiles:   NotImplementedYet

    employing the <xarray.Dataset> Size: 190MB
    Dimensions:                   (location: 5, time: 373, slope: 5,
                                   realization: 1, layer: 33)
    Coordinates:
      * location                  (location) <U5 100B 'VIR1A' 'VIR2A' ... 'VIR5A'
      * time                      (time) datetime64[ns] 3kB 2024-01-18 ... 2024-0...
      * slope                     (slope) int64 40B 0 1 2 3 4
      * realization               (realization) int64 8B 0
      * layer                     (layer) int64 264B 0 1 2 3 4 5 ... 28 29 30 31 32
        inclination               (location, slope) int64 200B 0 38 38 ... 38 38 38
        azimuth                   (location, slope) int64 200B 0 0 90 ... 90 180 270
        latitude                  (location) float64 40B 47.15 47.44 ... 47.44 47.37
        longitude                 (location) float64 40B 11.19 11.29 11.3 11.39 11.5
        altitude                  (location) float64 40B 2.372e+03 ... 2.066e+03
        z                         (location, time, slope, realization, layer) float32 1MB ...
    Data variables: (12/90)
        air_content               (location, time, slope, realization, layer) float32 1MB ...
        bond_size                 (location, time, slope, realization, layer) float32 1MB ...
        coord_number              (location, time, slope, realization, layer) float32 1MB ...
        dendricity                (location, time, slope, realization, layer) float32 1MB ...
        density                   (location, time, slope, realization, layer) float32 1MB ...
        element_ID                (location, time, slope, realization, layer) float32 1MB ...
        ...                        ...
        Ss                        (location, slope, realization, time, layer) float64 2MB ...
        zS4                       (location, slope, realization, time, layer) float64 2MB ...
        S4                        (location, slope, realization, time, layer) float64 2MB ...
        zS5                       (location, slope, realization, time, layer) float64 2MB ...
        S5                        (location, slope, realization, time, layer) float64 2MB ...
        HS                        (location, time, slope, realization, layer) float32 1MB ...
    Attributes:
        Conventions:  CF-1.8
        title:        Snowpack Profile for VIR1A
        source:       SNOWPACK model output (.pro file)
        history:      2025-11-13T18:54:08.878715+00:00: Created by xsnow.
        crs:          EPSG:4326

## Part 2: Filtering Data with Conditions

You can filter data based on conditions (e.g., "show me only dense layers"). This is done with `.where()`:

**What you'll see:** The code below creates filtered datasets containing only layers that meet specific criteria (dense snow, cold temperatures). These filtered datasets will have fewer layers than the original.

**Now You Try**: After running the examples, try filtering for:
- Layers with temperature between -5°C and 0°C (hint: use `&` for "and", `|` for "or")
- Layers with density greater than 250 kg/m³


In [57]:
# Find layers with temperature < -5°C
cold_mask = ds['temperature'] < -5
cold_layers = ds.where(cold_mask, drop=True)
print(cold_layers)

<xsnowDataset>
  Locations:  5
  Timestamps: 271 (2024-01-18--2024-02-02)
  Profiles:   NotImplementedYet

    employing the <xarray.Dataset> Size: 138MB
    Dimensions:                   (location: 5, time: 271, slope: 5,
                                   realization: 1, layer: 33)
    Coordinates:
      * location                  (location) <U5 100B 'VIR1A' 'VIR2A' ... 'VIR5A'
      * time                      (time) datetime64[ns] 2kB 2024-01-18T15:00:00 ....
      * slope                     (slope) int64 40B 0 1 2 3 4
      * realization               (realization) int64 8B 0
      * layer                     (layer) int64 264B 0 1 2 3 4 5 ... 28 29 30 31 32
        inclination               (location, slope) int64 200B 0 38 38 ... 38 38 38
        azimuth                   (location, slope) int64 200B 0 0 90 ... 90 180 270
        latitude                  (location) float64 40B 47.15 47.44 ... 47.44 47.37
        longitude                 (location) float64 40B 11.19 11.29 11.

In [58]:
# Find layers with density > 300 kg/m³
dense_mask = ds['density'] > 300
dense_layers = ds.where(dense_mask, drop=True)
print(dense_layers)


<xsnowDataset>
  Locations:  5
  Timestamps: 217 (2024-01-24--2024-02-02)
  Profiles:   NotImplementedYet

    employing the <xarray.Dataset> Size: 60MB
    Dimensions:                   (location: 5, time: 217, slope: 5,
                                   realization: 1, layer: 18)
    Coordinates:
      * location                  (location) <U5 100B 'VIR1A' 'VIR2A' ... 'VIR5A'
      * time                      (time) datetime64[ns] 2kB 2024-01-24T06:00:00 ....
      * slope                     (slope) int64 40B 0 1 2 3 4
      * realization               (realization) int64 8B 0
      * layer                     (layer) int64 144B 0 1 2 3 4 5 ... 13 14 15 16 17
        inclination               (location, slope) int64 200B 0 38 38 ... 38 38 38
        azimuth                   (location, slope) int64 200B 0 0 90 ... 90 180 270
        latitude                  (location) float64 40B 47.15 47.44 ... 47.44 47.37
        longitude                 (location) float64 40B 11.19 11.29 11.3

In [59]:
# Find layers with density > 300 kg/m^3 and grain_size < 0.5"
dense_and_small_mask = (ds['density'] > 300) & (ds['grain_size'] < 0.5)
dense_and_small = ds.where(dense_and_small_mask, drop=True)
print(dense_and_small)

<xsnowDataset>
  Locations:  2
  Timestamps: 120 (2024-01-25--2024-02-02)
  Profiles:   NotImplementedYet

    employing the <xarray.Dataset> Size: 5MB
    Dimensions:                   (location: 2, time: 120, slope: 3,
                                   realization: 1, layer: 11)
    Coordinates:
      * location                  (location) <U5 40B 'VIR4A' 'VIR5A'
      * time                      (time) datetime64[ns] 960B 2024-01-25T01:00:00 ...
      * slope                     (slope) int64 24B 0 3 4
      * realization               (realization) int64 8B 0
      * layer                     (layer) int64 88B 3 4 5 7 8 9 12 13 14 15 17
        inclination               (location, slope) int64 48B 0 38 38 0 38 38
        azimuth                   (location, slope) int64 48B 0 180 270 0 180 270
        latitude                  (location) float64 16B 47.44 47.37
        longitude                 (location) float64 16B 11.39 11.5
        altitude                  (location) float64 

In [63]:
# Which locations have layers that are dense and small?
dense_and_small_locations = dense_and_small.coords['location'].values
print(dense_and_small_locations)

# What azimuth at these locations?
print(dense_and_small.coords['azimuth'].values)

['VIR4A' 'VIR5A']
[[  0 180 270]
 [  0 180 270]]


## Part 3: Computing Profile-Level Summaries

Often you want to summarize layer-level data into a single value per profile. For example, "what's the average density in each profile?"

**What you'll see:** The code below computes mean, max, and min density across all layers for each profile. The result is a new variable with the `layer` dimension removed - it's now a profile-level variable (one value per profile per time step).


In [79]:
# Compute mean density across all layers for each profile
mean_density = ds['density'].mean(dim='layer')

# Other useful aggregations
max_density = ds['density'].max(dim='layer')  # Maximum density in profile
min_density = ds['density'].min(dim='layer')   # Minimum density in profile
std_density = ds['density'].std(dim='layer')   # Standard deviation



In [81]:
# Find the average grain size across all layers for each profile
mean_grain_size = ds['grain_size'].mean(dim='layer')

# Find the maximum, minimum, and std grain size per profile across all layers
max_grain_size = ds['grain_size'].max(dim='layer')
min_grain_size = ds['grain_size'].min(dim='layer')
std_grain_size = ds['grain_size'].std(dim='layer')


In [82]:
# Compute mean over time (average profile)
mean_density_over_time = ds['density'].mean(dim='time')

# Compute time series of profile-level variables
hs_series = ds['HS'].isel(location=0, slope=0, realization=0)

# Compute change over time (difference between consecutive time steps)
hs_change = ds['HS'].diff(dim='time')
print(f"Mean snow height change: {hs_change.mean().values:.4f} m/time step")


Mean snow height change: 0.0624 m/time step


## Summary

✅ **What we learned:**

1. **Selecting data**: `.sel()` for labels, `.isel()` for positions
2. **Filtering**: `.where()` with conditions to find specific layers/profiles
3. **Aggregations**: `.mean()`, `.max()`, `.min()`, `.sum()` across dimensions


## Key Operations Cheat Sheet

```python
# Selection
ds.sel(location="VIR1A", time="2024-02-01")  # By label
ds.isel(time=0, layer=0)                      # By position

# Filtering
ds.where(ds['density'] > 300, drop=True)      # Condition

# Aggregations
ds['density'].mean(dim='layer')               # Mean over layers
ds['HS'].max(dim='time')                      # Max over time

# Time operations
ds['HS'].diff(dim='time')                     # Change over time
ds.resample(time='1D').mean()                 # Resample

# NumPy
ds['density'].values                          # Get array
```
